In [94]:
import pandas as pd
from sodapy import Socrata
import requests
import plotly

records = str(10000)
def get_trees(records):
    client = Socrata("data.cityofnewyork.us", None)
    results = client.get("uvpi-gqnh", 
                         limit=records, 
                         order='tree_id')

    # Convert to pandas DataFrame
    trees = (pd.DataFrame.from_records(results).astype({'stump_diam':'int','latitude': 'float64', 'longitude': 'float64'})
            .replace('Good','1 - Good')
            .replace('Fair','2 - Fair')
            .replace('Poor','3 - Poor')
            .replace('Stump/Dead','4 - Stump/Dead'))
    
    trees.health = trees.health.fillna('Stump/Dead')
    return trees

df = get_trees(records)
df

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,3,501451,2015-05-19T00:00:00.000,24,0,OnCurb,Alive,1 - Good,Quercus palustris,pin oak,...,2009300,New York,40.816467,-73.889175,1014925.946,236757.4196,17,93,2006497,2027630075
1,4,501451,2015-05-19T00:00:00.000,20,0,OnCurb,Alive,1 - Good,Quercus palustris,pin oak,...,2009300,New York,40.816434,-73.889595,1014809.582,236745.2369,17,93,2006002,2027400100
2,7,501911,2015-05-19T00:00:00.000,3,0,OnCurb,Alive,3 - Poor,Betula pendula,silver birch,...,2009300,New York,40.814092,-73.889123,1014941.504,235892.233,17,93,2006623,2027650198
3,8,501911,2015-05-19T00:00:00.000,3,0,OnCurb,Alive,3 - Poor,Betula pendula,silver birch,...,2009300,New York,40.814100,-73.889070,1014956.216,235895.1546,17,93,2006623,2027650198
4,9,501911,2015-05-19T00:00:00.000,4,0,OnCurb,Alive,3 - Poor,Betula pendula,silver birch,...,2009300,New York,40.814107,-73.889021,1014969.752,235897.8422,17,93,2006623,2027650198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,11447,345010,2015-06-06T00:00:00.000,4,0,OnCurb,Alive,1 - Good,Syringa reticulata,Japanese tree lilac,...,4054700,New York,40.704823,-73.906896,1010064.08,196076.1161,34,547,4082912,4034670057
9996,11448,345010,2015-06-06T00:00:00.000,10,0,OnCurb,Alive,1 - Good,Fraxinus pennsylvanica,green ash,...,4054700,New York,40.704687,-73.907035,1010025.598,196026.4876,34,547,4082914,4034670060
9997,11449,345010,2015-06-06T00:00:00.000,8,0,OnCurb,Alive,1 - Good,Acer platanoides,Norway maple,...,4054700,New York,40.704605,-73.907118,1010002.496,195996.695,34,547,4082916,4034670064
9998,11450,345010,2015-06-06T00:00:00.000,4,0,OnCurb,Alive,1 - Good,Cornus florida,flowering dogwood,...,4054700,New York,40.704554,-73.907171,1009987.973,195977.966,34,547,4082917,4034670066


1. What proportion of trees are in good, fair, or poor health according to the 'health' variable?

In [28]:
import plotly.graph_objects as go
variable = 'curb_loc'
variable_val = ['OnCurb']
data = (pd.DataFrame(df[df[variable].isin(variable_val)]
                     .groupby(by='health')
                    .size()/len(df))
        .rename(columns={0:'proportion'})
        .reset_index()
        .replace('Good','1 - Good')
        .replace('Fair','2 - Fair')
        .replace('Poor','3 - Poor')
        .replace('Stump/Dead','4 - Stump/Dead')
        .sort_values(by='health')
        )
fig = go.Figure(data=[go.Table(
    header=dict(values=['Health', 'Proportion'],
                align='left'),
    cells=dict(values=[data.health, # 1st column
                       data.proportion], # 2nd column
               align='left'))
])

fig.update_layout(width=500, height=300)
fig.show()

In [98]:
data_heat_map = pd.pivot_table(df, index='health', columns='steward', values = 'tree_id', aggfunc=len)

In [166]:
str(tuple(df.spc_common.dropna().unique()))

'(\'pin oak\', \'silver birch\', \'littleleaf linden\', \'honeylocust\', \'eastern redbud\', \'Japanese zelkova\', \'red maple\', \'London planetree\', \'green ash\', \'Amur maackia\', \'Japanese tree lilac\', \'blackgum\', \'ginkgo\', \'Chinese elm\', \'northern red oak\', \'common hackberry\', \'cherry\', \'serviceberry\', \'American elm\', "\'Schubert\' chokecherry", \'swamp white oak\', \'silver maple\', \'katsura tree\', \'dawn redwood\', \'silver linden\', \'willow oak\', \'black walnut\', \'Norway maple\', \'sycamore maple\', \'white oak\', \'Callery pear\', \'Sophora\', \'hedge maple\', \'American linden\', \'sugar maple\', \'Siberian elm\', \'Japanese snowbell\', \'American hornbeam\', \'crab apple\', \'maple\', \'Persian ironwood\', \'sweetgum\', \'white ash\', \'Atlantic white cedar\', \'hawthorn\', \'Japanese hornbeam\', \'sassafras\', \'mulberry\', \'black locust\', \'empress tree\', \'golden raintree\', \'purple-leaf plum\', \'southern red oak\', \'English oak\', \'white 

In [195]:
url = "https://data.cityofnewyork.us/resource/uvpi-gqnh.json?$limit={}&$where=spc_common in{}".format(records,
                                                                                                      tuple(df.spc_common.dropna().unique()))

df = pd.read_json(requests.get(url).text)
df

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,167830,302238,2015-08-23,4,0,OnCurb,Alive,Fair,Fagus grandifolia,American beech,...,4011100,New York,40.778726,-73.905407,1.010448e+06,223001.8890,22.0,111.0,4016694.0,4.008280e+09
1,183730,415396,2015-08-29,5,0,OnCurb,Alive,Good,Fagus grandifolia,American beech,...,5013800,New York,40.568042,-74.128503,9.485481e+05,146255.8394,50.0,138.0,5059793.0,5.043640e+09
2,146024,211726,2015-08-15,2,0,OnCurb,Alive,Good,Fagus grandifolia,American beech,...,3030100,New York,40.677102,-73.912761,1.008448e+06,185974.8154,41.0,301.0,3042518.0,3.015650e+09
3,165501,206062,2015-08-22,4,0,OnCurb,Alive,Good,Fagus grandifolia,American beech,...,3018200,New York,40.612595,-74.002436,9.835736e+05,162461.1105,43.0,182.0,3163231.0,3.062830e+09
4,165503,206062,2015-08-22,2,0,OnCurb,Alive,Poor,Fagus grandifolia,American beech,...,3018200,New York,40.612814,-74.002798,9.834731e+05,162541.0300,43.0,182.0,3163226.0,3.062830e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,429304,202545,2015-11-04,2,0,OnCurb,Alive,Good,Carpinus caroliniana,American hornbeam,...,3114201,New York,40.681183,-73.894081,1.013628e+06,187467.6222,37.0,114201.0,3080646.0,3.034880e+09
9996,363149,209522,2015-10-22,6,0,OnCurb,Alive,Good,Carpinus caroliniana,American hornbeam,...,3059401,New York,40.597129,-73.947016,9.989641e+05,156830.9085,48.0,59401.0,3201240.0,3.073820e+09
9997,347448,309086,2015-10-19,10,0,OnCurb,Alive,Good,Carpinus caroliniana,American hornbeam,...,4015400,New York,40.692395,-73.822075,1.033591e+06,191584.6805,28.0,154.0,4199864.0,4.094650e+09
9998,352793,505885,2015-10-20,2,0,OnCurb,Alive,Fair,Carpinus caroliniana,American hornbeam,...,2025500,New York,40.862053,-73.909199,1.009366e+06,253359.6928,14.0,255.0,2014962.0,2.032250e+09


In [143]:
sum(list(data_heat_map.fillna(0).iloc[:,0].values))

2198.0

In [142]:
list(data_heat_map.fillna(0).iloc[0].values)/sum(list(data_heat_map.fillna(0).iloc[:,0].values))

array([0.71565059, 0.09417652, 0.01137398, 2.30709736])

In [145]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
                   z=[list(data_heat_map.fillna(0).iloc[0].values), 
                      list(data_heat_map.fillna(0).iloc[1].values),
                      list(data_heat_map.fillna(0).iloc[2].values)],
                   x=list(data_heat_map.columns),
                   y=list(data_heat_map.index.values))).update_layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Amount of Stewards",
            font=dict(
                size=11,
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Tree Health",
            font=dict(
                size=11,
            )
        )
    )
)
fig.show()

In [10]:

token = "pk.eyJ1IjoibWtpdmVuc29uIiwiYSI6ImNrMjljbGkxcjExamwzbW9qeGR0aHk1cDcifQ.PCzDqnDF1D_WLD6r9y34jA"

import plotly.express as px

fig = px.scatter_mapbox(df, lat='latitude', lon='longitude', hover_name='address', color = 'health',
                        hover_data=['status', 'health'],
                        zoom=9, height=500).update_layout(mapbox_style="dark", mapbox_accesstoken=token, margin={"r":0,"t":0,"l":0,"b":0})

fig
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


2. Are stewards (steward activity measured by the 'steward' variable) having an inpact on the health of trees?